In [2]:
import smtplib
from email.mime.text import MIMEText

# Email configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail, change if using Outlook or another service
SMTP_PORT = 465  # SSL port (or use 587 for TLS)
SENDER_EMAIL = "abhaymathur1000@gmail.com"  # Replace with your email
SENDER_PASSWORD = "blom axpb quot zcdv"  # Replace with your email password
RECEIVER_EMAIL = "abhaymathur1000@gmail.com"  # Your email (or another recipient)

def send_email(subject, body=""):
    """Sends an email notification"""
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECEIVER_EMAIL

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")



# Notify when the script completes
# send_email("✅ Script Completed", "Your Python script has finished running.")


# Imports

In [3]:
# Import Libraries
import os
import time
import datetime
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer, get_linear_schedule_with_warmup
from IPython.display import display, clear_output
import dask.dataframe as dd
import numpy as np

/home/abhay/Documents/Repo/abhayma1000/DS3010FinalProject/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-28 11:57:31.121589: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-28 11:57:31.271132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740761851.334309  540001 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740761851.3

# Model

In [ ]:
class BertClassifier:
    def __init__(self, data_path, tensor_path=None, model_name="bert-base-uncased", num_labels=None, batch_size=32, epochs=1, n_part=1, subset=False):
        """
        Initializes the BertClassifier.

        Args:
            data_path (str): Path to the directory containing the Parquet files.
            model_name (str, optional): Name of the BERT model to use. Defaults to "bert-base-uncased".
            num_labels (int, optional): Number of output labels. If None, it will be inferred from the data. Defaults to None.
            batch_size (int, optional): Batch size for training and evaluation. Defaults to 32.
            epochs (int, optional): Number of training epochs. Defaults to 1.
        """
        self.data_path = data_path
        self.model_name = model_name
        self.batch_size = batch_size
        self.epochs = epochs
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Load data and preprocess
        self.df_dask = dd.read_parquet(os.path.join(self.data_path, "*.parquet"), engine="pyarrow", npartitions=n_part)
        
        if subset:
            self.df_dask = self.df_dask.sample(frac=1. / n_part, random_state=42)
        print("Read parquet: ", self.df_dask.columns)

        self.tokenizer = BertTokenizer.from_pretrained(self.model_name, do_lower_case=True)
        print("Created Tokenizer")

        if tensor_path is not None and os.path.exists(tensor_path):
            tensors = torch.load(tensor_path)
            self.inputs = tensors["inputs"]
            self.attention_masks = tensors["attention_masks"]
            self.outputs = tensors["outputs"]
        else:
            self.inputs, self.attention_masks, self.outputs = self._preprocess_data(num_labels, tensor_save_path=tensor_path)
        print("Preprocessed data")
        self.train_dataloader, self.validation_dataloader, self.test_dataloader = self._create_dataloaders()
        print("Created dataloaders")

        # Initialize model, optimizer, and scheduler
        self.model = BertForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_labels,
            output_attentions=False,
            output_hidden_states=False,
            problem_type="multi_label_classification"
        ).to(self.device)
        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)
        self.scheduler = self._get_scheduler()
        self.criterion = nn.BCEWithLogitsLoss()

    def _preprocess_data(self, num_labels, tensor_save_path=None):
        """Preprocesses the data."""
        inputs = self.df_dask["text"].compute().values
        print("Inputs: ", inputs)
        print("Inputs Shape: ", inputs.shape)

        # One-Hot Encode
        mlb = MultiLabelBinarizer()
        outputs = mlb.fit_transform(self.df_dask['categories'].compute()).astype(float)

        # Total Number of Unique Labels
        if num_labels is None:
            num_labels = outputs.shape[1]

        input_ids = []
        attention_masks = []

        print('Start Tokenization')
        start_time = time.time()

        # For every review...
        for rev in inputs:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = self.tokenizer.encode_plus(
                rev,  # Sentence to encode.
                add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
                padding='max_length',
                max_length=200,  # Pad & truncate all sentences. Reviews over 512 so truncate there for now.
                pad_to_max_length=True,
                return_attention_mask=True,  # Construct attn. masks.
                return_tensors='pt',  # Return pytorch tensors.
                truncation=True  # Truncate at max_length
            )

            # Add the encoded sentence to the list.
            input_ids.append(encoded_dict['input_ids'])

            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Tokenizer Encoding: {elapsed_time:.4f} seconds")

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        outputs = torch.tensor(outputs).clone().detach()

        if tensor_save_path is not None:
            torch.save({"inputs": input_ids, "attention_masks": attention_masks, "outputs": outputs}, tensor_save_path)

        return input_ids, attention_masks, outputs

    def _create_dataloaders(self):
        """Creates dataloaders for training, validation, and testing."""
        # Combine the training inputs into a TensorDataset.
        dataset = TensorDataset(self.inputs, self.attention_masks, self.outputs)


        # Create a 75-15-10 train-validation-test split.

        # Calculate the number of samples to include in each set.
        train_size = int(0.75 * len(dataset))
        val_size_b4_test = int(len(dataset) - train_size)

        # Divide the dataset by randomly selecting samples.
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size_b4_test])

        val_size = int(0.15 * len(dataset))
        test_size = int(len(val_dataset) - val_size)
        val_dataset, test_dataset = random_split(val_dataset, [val_size, test_size])

        print('{:>5,} training samples'.format(train_size))
        print('{:>5,} validation samples'.format(val_size))
        print('{:>5,} testing samples'.format(test_size))

        # Create the DataLoaders for our training and validation sets.
        # We'll take training samples in random order.
        train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler=RandomSampler(train_dataset),  # Select batches randomly
            batch_size=self.batch_size  # Trains with this batch size.
        )

        # For validation the order doesn't matter, so we'll just read them sequentially.
        validation_dataloader = DataLoader(
            val_dataset,  # The validation samples.
            sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
            batch_size=self.batch_size  # Evaluate with this batch size.
        )

        test_dataloader = DataLoader(
            test_dataset,  # The validation samples.
            sampler=SequentialSampler(test_dataset),  # Pull out batches sequentially.
            batch_size=self.batch_size  # Evaluate with this batch size.
        )

        return train_dataloader, validation_dataloader, test_dataloader

    def _get_scheduler(self):
        """Creates the learning rate scheduler."""
        total_steps = len(self.train_dataloader) * self.epochs
        scheduler = get_linear_schedule_with_warmup(self.optimizer,
                                                    num_warmup_steps=0,  # Default value in run_glue.py
                                                    num_training_steps=total_steps)
        return scheduler

    def train(self):
        """Trains the model."""
        for e in range(self.epochs):
            print(f"\n===== Epoch {e + 1}/{self.epochs} =====")

            # training
            print("Training started ...")
            self._train_epoch(e)

            # validation testing
            print("Testing started ...")
            self._test_epoch(self.validation_dataloader, e)

    def _train_epoch(self, epoch):
        """Trains the model for one epoch."""
        self.model.train()
        train_loss = 0

        # Number of iterations equal to total train dataset / batch size
        for step, batch in enumerate(self.train_dataloader):
            # Print progress in epoch
            print(f"Progress: {step + 1}/{len(self.train_dataloader)}", end='\r')
            # Parse iterator tensor dataset for important information
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)
            output = self.model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)

            # Generate prediction
            self.optimizer.zero_grad()

            # Compute gradients and update weights
            loss = self.criterion(output.logits, b_labels)  # BCEWithLogitsLoss has sigmoid
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()

            # accumulate train loss
            train_loss += loss

        # print completed result
        print()
        print('Train Loss: %f' % (train_loss))
        return train_loss

    def test(self):
        """Evaluates the model on the test set."""
        return self._test_epoch(self.test_dataloader, self.epochs - 1)

    def _test_epoch(self, iterator, epoch):
        """Evaluates the model on the given dataloader."""
        self.model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for step, batch in enumerate(iterator):
                # Print progress in epoch
                print(f"Progress: {step + 1}/{len(iterator)}", end='\r')
                # Parse iterator tensor dataset for important information
                b_input_ids = batch[0].to(self.device)
                b_input_mask = batch[1].to(self.device)
                b_labels = batch[2].to(self.device)

                # generate prediction
                output = self.model(b_input_ids,
                                    token_type_ids=None,
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
                prob = output.logits.sigmoid()  # BCEWithLogitsLoss has sigmoid

                # record processed data count
                total += (b_labels.size(0) * b_labels.size(1))

                # take the index of the highest prob as prediction output
                THRESHOLD = 0.7
                prediction = prob.detach().clone()
                prediction[prediction > THRESHOLD] = 1
                prediction[prediction <= THRESHOLD] = 0
                correct += prediction.eq(b_labels).sum().item()

                print()

        # print completed result
        acc = 100. * correct / total
        print('Correct: %i   / Total: %i / Test Accuracy: %f' % (correct, total, acc))
        return acc

    def save(self, save_path):
        """Saves the trained model."""
        count = 0
        while os.path.exists(save_path + str(count)):
            count += 1

        self.model.save_pretrained(save_path + str(count))



In [8]:

data_path = 'data/lemm_stop_reviews_parquet'
classifier = BertClassifier(data_path)
classifier.train()
classifier.save(os.path.join('models', 'berttrained'))

send_email("✅ BERT Training complete", "")

Read parquet:  Index(['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny',
       'cool', 'text', 'date', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars_y', 'review_count',
       'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')
Created Tokenizer
Inputs:  <ArrowStringArray>
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

: 